In [1]:
import os 
import re
import json 
import pandas as pd

from datetime import datetime
from functools import reduce


In [2]:
dir = "data/location_data/"

In [3]:
mobility_all = pd.read_csv(dir + 'Global_Mobility_Report.csv')
epidem_all = pd.read_csv(dir + 'epidemiological_data.csv')
stringency_all = pd.read_csv(dir + 'OxCGRT_withnotes_2020.csv')

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
#mobility_all.head()
#epidem_all.head()
#stringency_all.head()

,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2019,continentExp,Cumulative_number_for_14_days_of_COVID-19_cases_per_100000
0,14/12/2020,14,12,2020,746,6,Afghanistan,AF,AFG,38041757.0,Asia,9.013779
1,13/12/2020,13,12,2020,298,9,Afghanistan,AF,AFG,38041757.0,Asia,7.052776
2,12/12/2020,12,12,2020,113,11,Afghanistan,AF,AFG,38041757.0,Asia,6.868768
3,11/12/2020,11,12,2020,63,10,Afghanistan,AF,AFG,38041757.0,Asia,7.134266
4,10/12/2020,10,12,2020,202,16,Afghanistan,AF,AFG,38041757.0,Asia,6.968658


In [7]:
#fixing the columns in mobility data 
loc_cols = ["country_region_code", "date",] #two columns determining location and one for date
baseline_cols = [col for col in mobility_all.columns if 'baseline' in col] #getting all the cool columns 

mob_cols = loc_cols + baseline_cols #Making a list of the columns i want

print(mob_cols)

mobility = mobility_all[mob_cols].copy(deep=True) #retrieving cols 

#renaming for later merging
mobility.rename(columns = {'country_region_code':'alpha2'}, inplace = True) 

mobility.head()



['country_region_code', 'date', 'retail_and_recreation_percent_change_from_baseline', 'grocery_and_pharmacy_percent_change_from_baseline', 'parks_percent_change_from_baseline', 'transit_stations_percent_change_from_baseline', 'workplaces_percent_change_from_baseline', 'residential_percent_change_from_baseline']


,alpha2,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,AE,2020-02-15,0.0,4.0,5.0,0.0,2.0,1.0
1,AE,2020-02-16,1.0,4.0,4.0,1.0,2.0,1.0
2,AE,2020-02-17,-1.0,1.0,5.0,1.0,2.0,1.0
3,AE,2020-02-18,-2.0,1.0,5.0,0.0,2.0,1.0
4,AE,2020-02-19,-2.0,0.0,4.0,-1.0,2.0,1.0


In [8]:
#fixing the columns in epidem data 
epi_cols = ["countryterritoryCode", "dateRep", "cases", "deaths", "Cumulative_number_for_14_days_of_COVID-19_cases_per_100000"] #the cols i want

epidem = epidem_all[epi_cols].copy(deep=True) #retrieving them
epidem.rename(columns = {'countryterritoryCode':'alpha3', 'dateRep':'date'}, inplace = True) 

epidem['date'] = pd.to_datetime(epidem['date'])

epidem.head()

,alpha3,date,cases,deaths,Cumulative_number_for_14_days_of_COVID-19_cases_per_100000
0,AFG,2020-12-14,746,6,9.013779
1,AFG,2020-12-13,298,9,7.052776
2,AFG,2020-12-12,113,11,6.868768
3,AFG,2020-11-12,63,10,7.134266
4,AFG,2020-10-12,202,16,6.968658


In [9]:
#fixing the columns in oxford data 
oxf_cols = ["CountryCode", "Date", "StringencyIndex"] #the cols i want

stringency = stringency_all[oxf_cols].copy(deep=True) #retrieving them

stringency.rename(columns = {'CountryCode':'alpha3', 'Date': 'date'}, inplace = True) 

stringency['date'] = stringency['date'].astype(str)

stringency['date'] = pd.to_datetime(stringency['date'])

stringency.head()

,alpha3,date,StringencyIndex
0,ABW,2020-01-01,0.0
1,ABW,2020-01-02,0.0
2,ABW,2020-01-03,0.0
3,ABW,2020-01-04,0.0
4,ABW,2020-01-05,0.0


In [10]:
#exporting

mobility.to_csv("mobility.csv", index=False) #saving as a csv-file
stringency.to_csv("stringency.csv", index=False) #saving as a csv-file
epidem.to_csv("epidem.csv", index=False) #saving as a csv-file